In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
set_seed = 46
np.random.seed(set_seed)
tf.random.set_seed(set_seed)

In [3]:
df = pd.read_csv("ModifiedFER2013.csv", index_col = False)

In [4]:
px = df["pixel"]

In [5]:
spx = px.str.split()

In [6]:
X=[]
for i in range(len(spx)):
    s = np.array(spx[i]).reshape(48, 48).astype('float32')
    X.append(s/255)

In [7]:
X = np.array(X)

In [8]:
X.shape

(29689, 48, 48)

In [9]:
label = np.array(df["type"])

In [10]:
label.shape

(29689,)

In [11]:
y = np.array(df["emotion"])

In [12]:
X_train, X_valid, X_test = X[:24291], X[24291:26990], X[26990:]
y_train, y_valid, y_test = y[:24291], y[24291:26990], y[26990:]

In [ ]:
batch_size = 64
shapes = ((batch_size, 48, 48), (batch_size, 6))
initializer = keras.initializers.GlorotUniform(seed=set_seed)
activation =  keras.activations.softmax
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-5,
            decay_steps=24291 // batch_size * 10, decay_rate=0.95)       
number_of_classes = 6        

inputs = keras.Input(shape=(48, 48, 1))
conv1 = keras.layers.Conv2D(128,(3, 3), padding='same', activation='relu')(inputs)
do1 = keras.layers.Dropout(0.5, seed=set_seed)(conv1)         
conv2 = keras.layers.Conv2D(64,(3, 3), padding='same', activation='relu')(do1)
do2 = keras.layers.Dropout(0.5, seed=set_seed)(conv2)         
conv3 = keras.layers.Conv2D(32,(3, 3), padding='same', activation='relu')(do2)
do3 = keras.layers.Dropout(0.5, seed=set_seed)(conv3)       
flatten = keras.layers.Flatten()(do3)    
outputs = keras.layers.Dense(number_of_classes, kernel_initializer=initializer, activation=activation)(flatten) 
model = keras.Model(inputs, outputs)             
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

es_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                          epochs=1000, callbacks=[es_callback])

Epoch 1/1000
760/760 [==============================] - 38s 36ms/step - loss: 1.6499 - accuracy: 0.3004 - val_loss: 1.6574 - val_accuracy: 0.3201
Epoch 2/1000
760/760 [==============================] - 27s 35ms/step - loss: 1.6270 - accuracy: 0.3066 - val_loss: 1.6307 - val_accuracy: 0.3235
Epoch 3/1000
760/760 [==============================] - 27s 35ms/step - loss: 1.6140 - accuracy: 0.3122 - val_loss: 1.5977 - val_accuracy: 0.3409
Epoch 4/1000
760/760 [==============================] - 27s 35ms/step - loss: 1.5956 - accuracy: 0.3288 - val_loss: 1.5884 - val_accuracy: 0.3520
Epoch 5/1000
760/760 [==============================] - 27s 35ms/step - loss: 1.5698 - accuracy: 0.3500 - val_loss: 1.5494 - val_accuracy: 0.3853
Epoch 6/1000
760/760 [==============================] - 26s 35ms/step - loss: 1.5456 - accuracy: 0.3673 - val_loss: 1.5542 - val_accuracy: 0.3642
Epoch 7/1000
760/760 [==============================] - 26s 34ms/step - loss: 1.5221 - accuracy: 0.3830 - val_loss: 1.5017 -